# Data management

In this notebook, we will work with the following:

- Reading and cleaning data with `pandas`.
- Web scraping.
- API access.
- Merging and querying with `pandas`.

In [1]:
import urllib

import pandas as pd
from bs4 import BeautifulSoup
from nytimesarticle import articleAPI

# Reading and cleaning data

`pandas` supports a number of formats that we often find ourselves using.
For example, I often use data in the Stata `dta` and SAS `sas7bdat` formats.
In particular, if you find yourself putting full datasets from WRDS (especially the ones that are not accessible with web forms), you will end up using the SAS format.

`pandas` also handles formats like Excel `xlsx`, comma separated values `csv` (and, indeed, nearly any delimited file), and fixed width data.
The acquisition database, SDC Platinum, has a somewhat unreliable Excel output feature, and the `pandas` fixed width format reader takes nearly all of the pain out of reading in data exported that way.

Note: `pandas` can also write many of the formats that it can read.
A notable exception is `sas7bdat` because it is proprietary and undocumented.
The reader was written with some clever reverse engineering, but writing a valid file is difficult and probably not coming in the future (see [Github issue](https://github.com/pandas-dev/pandas/issues/13031)).
An easy workaround is using the SAS open format `xpt` or `csv`.

In [2]:
# Stata data
firmyear = pd.read_stata('./data/firmyear.dta')
firmyear.head()

,count_of_employees,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017


You are likely familiar with a number of data cleaning issues.
However, you may not yet know how to map on what you know in another program to Python.
The pandas documentation has a number of comparison references, including [R](https://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html), [Stata](https://pandas.pydata.org/pandas-docs/stable/comparison_with_stata.html) and [SAS](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sas.html).

Some brief examples are below.

In [3]:
firmyear.dtypes

count_of_employees    object
name                  object
year                  object
dtype: object

In [4]:
firmyear['year'] = firmyear['year'].astype('int')
firmyear['count_of_employees'] = firmyear['count_of_employees'].astype('int')

# Note, a more general version would be:
# cols = firmyear.columns.drop('name')
# firmyear[cols] = firmyear[cols].apply(pd.to_numeric, errors='coerce')

In [5]:
firmyear

,count_of_employees,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017
5,76,Google,2018


In [6]:
firmyear.dtypes

count_of_employees     int64
name                  object
year                   int64
dtype: object

In [7]:
# An example of using dictionaries.
_COLUMNS = {
    'count_of_employees': 'size_emp'
}

firmyear = firmyear.rename(columns=_COLUMNS)

In [8]:
firmyear

,size_emp,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017
5,76,Google,2018


In [9]:
# We can do per-group things like calculating differences.
firmyear['size_emp_change'] = firmyear.groupby(firmyear['name'])['size_emp'].diff()

In [10]:
firmyear

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0
5,76,Google,2018,2.0


# Web scraping

One helpful way of gathering text data is web scraping.
We usually do this in three steps:

1. Retrieve the pages with information we want.
1. Extract the data from the pages.
1. Clean and save the resulting data.

Let's walk through an example of getting press releases from the [Microsoft website](https://news.microsoft.com/category/press-releases/).

I often prefer to work out of order as follows:

1. Figure out how to extract data from one page that has the data.
1. Then, figure out how to automate getting the pages of interest.
1. Run those pages through the procedure in step 1.
1. Clean and save.

This has the benefit of solving what is usually the hardest problem first.

## Important note

As you'll see, the difficulty ramps up a lot here.
Web scraping is easily a full day topic on its own.
Hence, I have two main goals for you:

1. Get a sense of the logic and the process in solving the problem. This is a good start if you want to learn it yourself.
1. Understand what is feasible and achievable. This helps whether you do it yourself or farm it out (and there's a ready talent pool for this).

## Read one page

This is the hardest part.

In [11]:
pr_url_1 = 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/'
pr_req_1 = urllib.request.Request(pr_url_1)
pr_page_1 = urllib.request.urlopen(pr_req_1).read()

In [12]:
pr_soup_1 = BeautifulSoup(pr_page_1)

In [13]:
pr_soup_1.find('meta', property='article:published_time')

<meta content="2018-10-04T13:00:35+00:00" property="article:published_time"/>

In [14]:
pr_soup_1.find('meta', property='article:published_time')['property']

'article:published_time'

In [15]:
pr_soup_1.find('meta', property='article:published_time')['content']

'2018-10-04T13:00:35+00:00'

In [16]:
# List of meta tags to get.
# Note: when in doubt, get everything you might possibly use.
#       It's easier to drop stuff than to re-scrape everything.

_METAS = [
    'article:published_time',
    'article:modified_time',
    'og:title',
    'og:description',
    'og:updated_time',
    'og:url',
    'article:section'
]

In [17]:
pr_data_1 = {}
for meta in _METAS:
    prop = pr_soup_1.find('meta', property=meta)['property']
    content = pr_soup_1.find('meta', property=meta)['content']
    pr_data_1.update({prop: content})

In [18]:
pr_data_1

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases'}

In [19]:
pr_soup_1.find('div', {'class': 'entry-content m-blog-content'}).find('h3').string

'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally'

In [20]:
pr_data_1['h3'] = pr_soup_1.find('div', 
                                 {'class': 'entry-content m-blog-content'}
                                ).find('h3').string

In [21]:
pr_data_1

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases',
 'h3': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the 

In [22]:
pr_soup_1.find('div', 
               {'class': 'entry-content m-blog-content'}
              ).find_all('p')

[<p><strong>REDMOND, Wash. </strong><strong>—</strong> <strong>Oct. 4, 2018</strong> <strong>—</strong> On Thursday, <a href="https://rdlcom.com/">Redline Communications</a> (TSX:RDL) and <a href="https://www.microsoft.com/en-us/">Microsoft Corp.</a> announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a leader in private wireless networks, will provide its Virtual Fiber™ radio technology in the TV White Space band to Microsoft Airband Initiative partners. Together, Redline and Microsoft’s partnership will help make broadband internet more affordable and accessible to unserved and underserved customers in rural areas in the United States and globally.</p>,
 <p>New cloud services and other technologies make broadband connectivity a necessity to start and grow a small business and to take advantage of advances in agriculture, telemedicine and education. It is a vital part of 21st century infrastructure. Yet, more than 19.4 m

In [23]:
# This is a little gnarly.
pr_data_1['body'] = '\n\n'.join(
                        [i.text for i in pr_soup_1.find(
                            'div', 
                            {'class': 'entry-content m-blog-content'}
                            ).find_all('p')])

In [24]:
pr_data_1

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases',
 'h3': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the 

# Automate our one page work.

This is fairly easy. We have the code for it already.
We just need to wrap it in a function.

**Note:** I'm using an `if` statement to check whether these properties exist, and guarding against the case where they don't.
I did this iteratively while building this content, because I noticed (from errors) that many press releases do noe have modification dates.

In [25]:
def get_data_from_soup(soup):
    data = {}
    for meta in _METAS:
        if soup.find('meta', property=meta) is not None:
            prop = soup.find('meta', property=meta)['property']
        if soup.find('meta', property=meta) is not None:
            content = soup.find('meta', property=meta)['content']
        if prop is not None and content is not None:
            data.update({prop: content})
    try:
        data['h3'] = soup.find('div', 
                               {'class': 'entry-content m-blog-content'}
                              ).find('h3').string
    except AttributeError:
        data['h3'] = ''
    
    data['body'] = '\n\n'.join(
                        [i.text for i in soup.find(
                            'div', 
                            {'class': 'entry-content m-blog-content'}
                            ).find_all('p')])
    
    return data

In [26]:
# Notice how easy this is once we make a function.
get_data_from_soup(pr_soup_1)

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases',
 'h3': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the 

## Read many pages

Now we need to get the URLs for all of the pages we want.

In [27]:
many_pr_url_1 = 'https://news.microsoft.com/category/press-releases/'
many_pr_req_1 = urllib.request.Request(many_pr_url_1)
many_pr_page_1 = urllib.request.urlopen(many_pr_req_1).read()
many_pr_soup_1 = BeautifulSoup(many_pr_page_1)

In [28]:
# Almost, but note the ones at the bottom.
many_pr_soup_1.find('section', id='primary').find_all('a')

[<a class="f-post-link c-heading-6 m-chevron" href="https://news.microsoft.com/2018/10/11/microsoft-announces-quarterly-earnings-release-date-37/" ms.title="Microsoft announces quarterly earnings release date" rel="bookmark">
 		Microsoft announces quarterly earnings release date	</a>,
 <a class="f-post-link c-heading-6 m-chevron" href="https://news.microsoft.com/2018/10/11/indiana-governor-eric-j-holcomb-the-markle-foundation-and-microsoft-announce-skillful-indiana-bringing-new-investment-partners-and-innovation-to-indianas-next-level-workforce-agenda/" ms.title="Indiana Governor Eric J. Holcomb, the Markle Foundation and Microsoft announce Skillful Indiana: Bringing new investment, partners and innovation to Indiana’s next level workforce agenda" rel="bookmark">
 		Indiana Governor Eric J. Holcomb, the Markle Foundation and Microsoft announce Skillful Indiana: Bringing new investment, partners and innovation to Indiana’s next level workforce agenda	</a>,
 <a class="f-post-link c-head

In [29]:
# Almost, but note the ones at the bottom.
articles = many_pr_soup_1.find('section', id='primary').find_all('article')
links = [i.find('a')['href'] for i in articles]
links

['https://news.microsoft.com/2018/10/11/microsoft-announces-quarterly-earnings-release-date-37/',
 'https://news.microsoft.com/2018/10/11/indiana-governor-eric-j-holcomb-the-markle-foundation-and-microsoft-announce-skillful-indiana-bringing-new-investment-partners-and-innovation-to-indianas-next-level-workforce-agenda/',
 'https://news.microsoft.com/2018/10/11/anil-kumbles-microsoft-ai-enabled-power-bat-partners-with-star-india-to-revolutionize-cricketing-experience-for-players-and-fans/',
 'https://news.microsoft.com/2018/10/10/microsoft-and-lake-country-satellite-roll-out-free-wi-fi-in-boydton-virginia/',
 'https://news.microsoft.com/2018/10/10/blackbaud-and-microsoft-expand-partnership-with-an-integrated-cloud-initiative-for-nonprofits/',
 'https://news.microsoft.com/2018/10/08/grab-forges-strategic-cloud-partnership-with-microsoft-to-drive-innovation-and-adoption-of-digital-services-across-southeast-asia/',
 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsof

In [30]:
many_pr_links_1 = links.copy()

## Automate getting links and data from each

In [31]:
# We need to turn links into soup objects a lot, so let's make a function.
def link_to_soup(link):
    req = urllib.request.Request(link)
    page = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(page)
    return soup
    
def get_links_from_link_page(link_page):
    soup = link_to_soup(link_page)
    articles = soup.find('section', id='primary').find_all('article')
    links = [i.find('a')['href'] for i in articles]
    return links

def get_data_from_links(links):
    data_list = []
    for link in links:
        soup = link_to_soup(link)
        data_list.append(get_data_from_soup(soup))
        
    return data_list


In [32]:
msft_prs = pd.DataFrame(get_data_from_links(many_pr_links_1))
msft_prs.to_csv('./data/msft_prs.csv', index=False)
msft_prs.head()

,article:modified_time,article:published_time,article:section,body,h3,og:description,og:title,og:updated_time,og:url
0,2018-10-12T00:36:34+00:00,2018-10-11T23:16:52+00:00,Press Releases,"REDMOND, Wash. — Oct. 11, 2018 — Microsoft Cor...",Microsoft to host earnings conference call web...,Microsoft to host earnings conference call web...,Microsoft announces quarterly earnings release...,2018-10-12T00:36:34+00:00,https://news.microsoft.com/2018/10/11/microsof...
1,2018-10-11T16:36:30+00:00,2018-10-11T14:00:25+00:00,Press Releases,"INDIANAPOLIS – Oct. 11, 2018 – Today, Eric J. ...","With support provided by Walmart, and in partn...","With support provided by Walmart, and in partn...","Indiana Governor Eric J. Holcomb, the Markle F...",2018-10-11T16:36:30+00:00,https://news.microsoft.com/2018/10/11/indiana-...
2,2018-10-11T07:05:46+00:00,2018-10-11T07:04:28+00:00,Press Releases,"MUMBAI, India — Oct. 11, 2018 — Microsoft Corp...",None,"Microsoft, Star India and Anil Kumble redefine...",Anil Kumble’s Microsoft AI-enabled Power Bat p...,2018-10-11T07:05:46+00:00,https://news.microsoft.com/2018/10/11/anil-kum...
3,2018-10-11T20:29:31+00:00,2018-10-10T18:00:22+00:00,Press Releases,"BOYDTON, Va. — Oct. 10, 2018 — Microsoft Corp....",,"BOYDTON, Va. — Oct. 10, 2018 — Microsoft Corp....",Microsoft and Lake Country Satellite roll out ...,2018-10-11T20:29:31+00:00,https://news.microsoft.com/2018/10/10/microsof...
4,NaN,2018-10-10T15:15:58+00:00,Press Releases,"Charleston, S.C. (October 10, 2018) — Blackbau...",Introduce first phase of initiative with joint...,Introduce first phase of initiative with joint...,Blackbaud and Microsoft expand partnership wit...,NaN,https://news.microsoft.com/2018/10/10/blackbau...


**Note**: for running time reasons, we're not going to make a multi-links-page version, but note that there's a next page link at the bottom of those pages that can be extracted to build that:

```html
<a href="/category/press-releases/page/2/?paged=3" 
   class="c-glyph x-hidden-focus" 
   aria-label="Go to next page" ms.title="Next Page">
```

# APIs

As we're about to see, it's really nice when sites help us out.

We'll be using the New York Times API, and you'll need a key to use it. You can get one [here](https://developer.nytimes.com/signup), and it will show up by email (very quickly).

In [33]:
# You will replace REDACTED with the key you got in email.
# It's probably obvious, but don't share it.
nyt_api = articleAPI('REDACTED')

msft_articles = nyt_api.search(q='Microsoft', begin_date=20180801)

In [34]:
# Trust me that the JSON for even a short period is very long,
# thus the immediate dataframe processing.
msft_nyt = pd.DataFrame(pd.io.json.json_normalize(msft_articles['response']['docs']))
msft_nyt.head()

,_id,byline.organization,byline.original,byline.person,document_type,headline.content_kicker,headline.kicker,headline.main,headline.name,headline.print_headline,...,print_page,pub_date,score,section_name,snippet,source,type_of_material,uri,web_url,word_count
0,5bbb75cb068401528a2df0f5,REUTERS,By REUTERS,[],article,None,None,Microsoft Testing New Game-Streaming Service,None,Microsoft Testing New Game-Streaming Service,...,NaN,2018-10-08T15:20:41+0000,297.65900,NaN,Microsoft Corp is testing a new game streaming...,Reuters,News,nyt://article/b5c2521b-0946-5436-adb1-62d71a73...,https://www.nytimes.com/reuters/2018/10/08/tec...,141
1,5bbbe59b068401528a2df1ef,REUTERS,By REUTERS,[],article,None,None,Microsoft to Invest in Southeast Asian Ride-Ha...,None,Microsoft to Invest in Southeast Asian Ride-Ha...,...,NaN,2018-10-08T23:17:43+0000,276.20792,NaN,Microsoft Corp is investing in Southeast Asian...,Reuters,News,nyt://article/f5290bfb-217c-5308-b763-2207b405...,https://www.nytimes.com/reuters/2018/10/08/bus...,327
2,5bbb4bb5068401528a2df0a8,REUTERS,By REUTERS,[],article,None,None,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,None,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,...,NaN,2018-10-08T12:21:08+0000,271.93472,NaN,U.S. software giant Microsoft is set to win un...,Reuters,News,nyt://article/928a254e-c6eb-5eca-896c-3556a5fa...,https://www.nytimes.com/reuters/2018/10/08/bus...,174
3,5bbcabf1068401528a2df379,REUTERS,By REUTERS,[],article,None,None,Microsoft Expands Cloud Service in Push for $1...,None,Microsoft Expands Cloud Service in Push for $1...,...,NaN,2018-10-09T13:23:59+0000,270.83615,NaN,Microsoft Corp said on Tuesday its expanded Az...,Reuters,News,nyt://article/94179203-ee24-5ce5-bb96-641ee251...,https://www.nytimes.com/reuters/2018/10/09/bus...,158
4,5bb295c6068401528a2ddeeb,REUTERS,By REUTERS,[],article,None,None,Microsoft Co-Founder Paul Allen Treated Again ...,None,Microsoft Co-Founder Paul Allen Treated Again ...,...,NaN,2018-10-01T21:46:45+0000,257.48560,NaN,Microsoft Corp co-founder Paul Allen said on M...,Reuters,News,nyt://article/2f19d5d8-3def-5d8d-9ba8-519385d2...,https://www.nytimes.com/reuters/2018/10/01/us/...,273


In [35]:
# While we're at it, let's add Microsoft's ticker.
# We'd usually add an identifier when getting query results.
msft_nyt['id_ticker'] = 'msft'

In [36]:
# We should also clean up the publication date.
type(msft_nyt['pub_date'][0])

str

In [37]:
msft_nyt['pub_date'] = pd.to_datetime(msft_nyt['pub_date'])
type(msft_nyt['pub_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [38]:
# We're going to look at this a bit later, so let's save it.
msft_nyt.to_csv('./data/msft_nyt.csv', index=False)

# Commercial databases

This is a more difficult topic to cover in a hands-on workshop (especially one with  openly-distributed materials) because of copyright.
They tend to sort out into a couple of types:

1. Those that come in tabular formats and simply require cleaning.
2. Others (like LexisNexis and Factiva) that come in semi-structured form and require extensive parsing. 

For the second type, it's best to either recruit one of the few coauthors with this skillset (and perhaps even a written or partially written implementation) or hiring a programmer or student who can write one.
It's worth noting that it's not all that hard to get a reasonable implementation written, but going from 90 percent parsing accuracy to 99 percent to 99.9+ percent is difficult, painstaking work.

With that in mind, feel free to ask questions about this topic during the workshop.
If time permits, I'll demonstrate some non-sharable stuff.

# Merging

Like other software, `pandas` is great and merging data, and it as some conveniences not found in most other software.

Let's work through a simple example to see it in action.

In [39]:
# Remember our firm year data.
firmyear.head()

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0


In [40]:
stock = pd.read_csv('./data/stock.csv')
stock.head()

,price,tic,yr
0,86.13,msft,2018
1,62.79,msft,2017
2,54.32,msft,2016


What we'd like to do is merge in those Microsoft stock prices from the beginning of those years.
It's a bit contrived for an example, but it mirrors a lot of real world work.

While we know that Microsoft's ticker is MSFT, there's no way for `pandas` to know that without help.
So, to help, we'll make a lookup table using a dictionary.

In [41]:
lookup = {
    'Microsoft': 'MSFT',
    'Google': 'GOOG'
}

In [42]:
firmyear['id_ticker'] = firmyear['name'].map(lookup)
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker
0,114,Microsoft,2016,NaN,MSFT
1,124,Microsoft,2017,10.0,MSFT
2,131,Microsoft,2018,7.0,MSFT
3,72,Google,2016,NaN,GOOG
4,74,Google,2017,2.0,GOOG


In [43]:
# Let's make that lowercase.
firmyear['id_ticker'] = firmyear['id_ticker'].str.lower()
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker
0,114,Microsoft,2016,NaN,msft
1,124,Microsoft,2017,10.0,msft
2,131,Microsoft,2018,7.0,msft
3,72,Google,2016,NaN,goog
4,74,Google,2017,2.0,goog


In Stata, we would have another problem, namely that our column names for merging do not match.
With `pandas`, that's not a problem.

In [44]:
firmyear = firmyear.merge(stock, how='left', 
                          left_on=['id_ticker', 'year'],
                          right_on=['tic', 'yr'],
                          validate='1:1')

In [45]:
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN
4,74,Google,2017,2.0,goog,NaN,NaN,NaN


# Querying

When working with content data, we often need to do some sort of a query to aggregate data that is interesting to us.

For example, let's add the average word count of articles from our NYT data to our firmyear data.
We're only going to have results for 2018, as that's all the data we pulled.

In [46]:
_AGG = {
    'word_count': ['mean', 'sum']
}

def query_docs(data, ticker, year):
    summary = (data[(data['id_ticker'] == ticker) & 
                    (data['pub_date'].dt.year == year)].agg(_AGG)
                     .T.reset_index(drop=True))
    summary['id_ticker'] = ticker
    summary['year'] = year
    summary = summary.rename(columns={'mean': 'wc_mean', 'sum': 'wc_sum'})
    return summary

In [47]:
results = pd.DataFrame()
for index, row in firmyear.loc[:, ['id_ticker', 'year']].iterrows():
    results = results.append(query_docs(msft_nyt, row['id_ticker'], row['year']))

In [48]:
results

,wc_mean,wc_sum,id_ticker,year
0,NaN,0.0,msft,2016
0,NaN,0.0,msft,2017
0,519.0,5190.0,msft,2018
0,NaN,0.0,goog,2016
0,NaN,0.0,goog,2017
0,NaN,0.0,goog,2018


In [49]:
firmyear = firmyear.merge(results, how='left',
                          on=['id_ticker', 'year'],
                          validate='1:1')
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr,wc_mean,wc_sum
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0,NaN,0.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0,NaN,0.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0,519.0,5190.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN,NaN,0.0
4,74,Google,2017,2.0,goog,NaN,NaN,NaN,NaN,0.0
